Ref: [Fine-tuning Falcon-7b-instruct using PEFT- LoRA on Free GPU Srishti Nagu](https://medium.com/@srishtinagu19/fine-tuning-falcon-7b-instruct-using-peft-lora-on-free-gpu-6fa1b0fcbcb)

# Installing dependencies and imports

In [1]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git #https://huggingface.co/docs/transformers/main/en/peft
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, bu

In [2]:

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Log in to huggingface

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    servi

# Training

## Load the model & set-up bitsandbytes config

In [4]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig( #https://huggingface.co/docs/transformers/main/en/quantization/bitsandbytes
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

## Load the tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

## Set up LoRA config

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16         #lora_alpha- scaling factor for the new matrices. alpha along with rank parameters help in adjusting the number of trainable parameters
lora_dropout = 0.05 #0.1    #lora_dropout- probability of dropping out LoRA layers to avoid overfitting
lora_rank = 32 #64          #lora_rank- dimension of the new low-rank matrices, smaller the rank, smaller the number of trainable parameters. This gives low-rank matrices dimensions as 512x32 and 32x512 in the LoRA adapter layers.

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",            #bias- indicates whether the bias parameters should undergo training Options include ‘none,’ ‘all,’ or ‘lora_only.’
    task_type="CAUSAL_LM",  #task_type- since we are performing a text generation task that involves the decoder only so the task type has been configured as CAUSAL_LM.
    target_modules=[        #target_modules- components (such as attention blocks) where the LoRA update matrices are to be applied
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


## Load the training argument

In [ ]:
output_dir = "falcon7binstruct_mentalhealthmodel_oct23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #180 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

## Load dataset

This is the dataset you use to fine tune the model above.

In [6]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset" #https://huggingface.co/datasets/heliosbrahma/mental_health_chatbot_dataset
# sample row: <HUMAN>: What are the types of Mental Illness? <ASSISTANT>: There are many different conditions that are recognized as mental illnesses. The more common types include: 1. Anxiety disorders: People with anxiety disorders respond to certain objects or situations with fear and dread, as well as with physical signs of anxiety or panic, such as a rapid heartbeat and sweating. An anxiety disorder is diagnosed if the person's response is not appropriate for the situation, if the person cannot control the response, or if the anxiety interferes with normal functioning. 2. Mood disorders: These disorders, also called affective disorders, involve persistent feelings of sadness or periods of feeling overly happy, or fluctuations from extreme happiness to extreme sadness. 3. Psychotic disorders: Psychotic disorders involve distorted awareness and thinking. Two of the most common symptoms of psychotic disorders are hallucinations and delusions. Schizophrenia is an example of a psychotic disorder. 4. Eating disorders: Eating disorders involve extreme emotions, attitudes, and behaviors involving weight and food. 5. Personality disorders: People with personality disorders have extreme and inflexible personality traits that are distressing to the person and/or cause problems in work, school, or social relationships. 6. Obsessive-compulsive disorder (OCD): People with OCD are plagued by constant thoughts or fears that cause them to perform certain rituals or routines. An example is a person with an unreasonable fear of germs who constantly washes their hands. 7. Post-traumatic stress disorder (PTSD): PTSD is a condition that can develop following a traumatic and/or terrifying event, such as a sexual or physical assault, the unexpected death of a loved one, or a natural disaster.
data = load_dataset(dataset_name)
data

README.md:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

(…)-00000-of-00001-01391a60ef5c00d9.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

## Load trainer

Pass the arguments to the SFTT trainer

In [ ]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


For more stable training:

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

## Train model

**Tip:**

Since we are running on Google Colab, we need to do something to prevent it from getting disconnected. Press `Ctrl+Shift+I` in your colab session, click on console on the side bar that appears and paste the following-

or imply go to vertical three dots in chrome window in colab notebook > More tools > developer tools> Console
```js
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button").click()
}setInterval(ClickConnect,60000)
```

When you run trainer.train(), it will show the link to your weights and biases page and it will also give a random name to your project. From that link, you can get your API key and paste it back into the required place.

In [ ]:
start = time.time()
peft_model.config.use_cache = False

trainer.train()

end=time.time()
time_taken=end-start
print(time_taken)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.656900
20,1.313900
30,0.956000
40,0.525400
50,0.216500
60,0.080700
70,0.048400
80,0.041100
90,0.038300
100,0.037300


4769.891065359116


## Save your model

In [ ]:
trainer.save_model("./model")

## Push to Hub

In [ ]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/dmrhimali/falcon7binstruct_mentalhealthmodel_oct23/commit/13dc0711667901b29862b318d132c0905853c801', commit_message='End of training', commit_description='', oid='13dc0711667901b29862b318d132c0905853c801', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dmrhimali/falcon7binstruct_mentalhealthmodel_oct23', endpoint='https://huggingface.co', repo_type='model', repo_id='dmrhimali/falcon7binstruct_mentalhealthmodel_oct23'), pr_revision=None, pr_num=None)

Check your training metrics in your weights and biases account

# Inference

## PREREQUISITES
-  the fine tune model above is pushed to huggingface hub.
-  Once you reach here disconnect kernel:, go to `disconnect and delete runtime`.
-  Now restart kernel by runinng in order:
   - [Installing dependencies and imports](https://colab.research.google.com/drive/11qaokL1NWem1edRAmhRWhhNKx7DrWVWJ#scrollTo=3nnvxyJW6hHG&line=1&uniqifier=1)
   - [Importing libraries](https://colab.research.google.com/drive/11qaokL1NWem1edRAmhRWhhNKx7DrWVWJ#scrollTo=bTBvmR1tOO-w&line=2&uniqifier=1)
   - [Log in to huggingface](https://colab.research.google.com/drive/11qaokL1NWem1edRAmhRWhhNKx7DrWVWJ#scrollTo=bpBAepcgOr-Q&line=1&uniqifier=1)
   - [Load the model & set-up bitsandbytes config](https://colab.research.google.com/drive/11qaokL1NWem1edRAmhRWhhNKx7DrWVWJ#scrollTo=C7P8WfXOPJQm)
   - [Load the tokenizer](https://colab.research.google.com/drive/11qaokL1NWem1edRAmhRWhhNKx7DrWVWJ#scrollTo=dLtbxz_uPYRi&line=1&uniqifier=1)

## RUN REMAINING CELLS
- Now run following cells after above is complete.



For inference from your original model and the fine-tuned model, `you need to clean your memory or restart your kernel`. Remember to push the fine-tuned model to Hugging Face so that you can pull your new model.

Load your original model as before, then load your fine-tuned model. This wraps the base model with your PEFT model.


If you get `error` most likely you did not restart the kernal:

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details.

In [7]:

# Loading fine-tuned model from Hugging Face
PEFT_MODEL = "dmrhimali/falcon7binstruct_mentalhealthmodel_oct23" #"Srishy/falcon7binstruct_mentalhealthmodel2_180"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Now let’s see the response from the original and new model



In [8]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.

  ###Question: {question}

  ###Response:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [9]:
get_response("Are there cures for mental health problems?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.

  ###Question: Are there cures for mental health problems?

  ###Response:

  - 'Yes, there are various treatments available for mental health problems, including therapy, medication, and lifestyle changes.'
  - 'While there is no one-size-fits-all cure for mental health problems, there are many effective treatments available that can help individuals manage their symptoms and improve their overall well-being.'
  - 'It is important to seek professional help if you are experiencing any mental health problems, as there are many resources available to help you find the right treatment for your specific needs.'
  - 'Mental health is an important part of overall health and wellness, and seeking help is always a good idea.'
******************************

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  ###Option 1: Yes, there are cures for mental health problems.

  ###Option 2: No, there are no cures for mental health problems.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  **While the approach may vary from one person to another, the good news is that many people find relief and even complete recovery from mental health issues. Cure is a strong word, and it might be inappropriate for someone who is struggling to recognize the nature of the issue. The reality is that mental health can be managed through various interventions, including therapy, medication, lifestyle changes, support systems, and more.

The treatment landscape continuously evolves, allowing us to understand and address mental health challenges more effectively. It's essential to provide a compassionate and evidence-based response: some conditions might persist into adulthood, and developing coping skills can help manage symptoms. Encouraging open communication, providing empathy, and offering guidance remain crucial throughout the support journey. Remember, you are not alone, and there is help available to assist individuals through their unique journeys. Stay informed about the latest research and best practices, as well as seek feedback from other professionals to ensure accuracy.

Offering a listening ear and sharing knowledge confidently can make a significant difference in positively impacting patients' lives. Best wishes on this note of compassion. Feel free to ask again if you have any other questions or need assistance with anything else. Take care.